In [2]:
import numpy as np

In [4]:
## Sigmoid activation

class Activation:
    def __init__(self, activation, activation_prime):
        self.input = None
        self.activation = activation
        self.derv_activation = activation_prime

    def forward(self, input_data):
        self.input = input_data
        return self.activation(input_data)

    def backward(self, output_grad, learning_rate=None):
        return np.multiply(self.derv_activation(self.input), output_grad)
    
class Sigmoid(Activation):
    def __init__(self):
        sigmoid = lambda z : 1/(1+np.exp(-z))
        sigmoid_prime = lambda z : (sigmoid(z)) * (1 - sigmoid(z))
        super().__init__(sigmoid, sigmoid_prime)

In [5]:
## Dense layer

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input_data):
        pass

    def backward(self, output_grad, learning_rate):
        pass

class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size, 1)
        # each row represents a neuron
        # each col represents that neurons weight for that connnection

    def forward(self, input_data):
        self.input = input_data
        return np.dot(self.weights, self.input) + self.biases
    
    def backward(self, output_grad, learning_rate):
        weight_grad = np.dot(output_grad, self.input.T)
        input_grad = np.dot(self.weights.T, output_grad)
        self.weights -= learning_rate * weight_grad
        self.biases -= learning_rate * output_grad
        return input_grad

In [7]:
## MSE Loss

class LossFn:
    def __init__(self):
        pass

    def loss(self, y_true, y_pred):
        pass

    def gradient(self, y_true, y_pred):
        pass

class MSE(LossFn):
    def loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)/2

    def gradient(self, y_true, y_pred):
        return (y_pred - y_true) / y_true.size

In [8]:
## Sequential architecture

class Sequential:
    def __init__(self, architecture, loss_fn):
        self.prediction = None
        self.achitecture = architecture
        self.loss_fn = loss_fn

    def add(self, layer):
        self.achitecture.append(layer)

    def predict(self, input_data):
        for layer in self.achitecture:
            output = layer.forward(input_data)
            input_data = output

        self.prediction = output
        return self.prediction
    
    def train(self, X, y, epochs, lr):
        for epoch in range(epochs):
            error = 0
            for X_train, y_train in zip(X, y):
                # forward pass
                y_pred = self.predict(X_train)

                # Error
                error += self.loss_fn.loss(y_train, y_pred)

                # Backward
                grad = self.loss_fn.gradient(y_train, y_pred)
                for layer in reversed(self.achitecture):
                    grad = layer.backward(grad, lr)

                error /= len(X_train)

            print(f"Completed {epoch+1}/{epochs} epoch, loss = {error}")

In [13]:
## XOR Data
X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
X = X.reshape(4, 2, 1)
print(X)
print()

y = np.array([[0],
              [1],
              [1],
              [0]])
y = y.reshape(4, 1, 1)
print(y)

[[[0]
  [0]]

 [[0]
  [1]]

 [[1]
  [0]]

 [[1]
  [1]]]

[[[0]]

 [[1]]

 [[1]]

 [[0]]]


In [14]:
xor_model = Sequential([
        Dense(2, 3),
        Sigmoid(),
        Dense(3, 1),
        Sigmoid()
    ],
    loss_fn=MSE())

In [16]:
xor_model.train(X, y, epochs=10000, lr=0.1)

Completed 1/10000 epoch, loss = 0.14751854059847463
Completed 2/10000 epoch, loss = 0.1463635863985649
Completed 3/10000 epoch, loss = 0.1451737791259171
Completed 4/10000 epoch, loss = 0.14395016126789903
Completed 5/10000 epoch, loss = 0.1426942156095536
Completed 6/10000 epoch, loss = 0.14140791687361096
Completed 7/10000 epoch, loss = 0.1400937808690358
Completed 8/10000 epoch, loss = 0.1387549084438891
Completed 9/10000 epoch, loss = 0.13739502102303452
Completed 10/10000 epoch, loss = 0.13601848407273032
Completed 11/10000 epoch, loss = 0.13463031454706104
Completed 12/10000 epoch, loss = 0.13323616832198767
Completed 13/10000 epoch, loss = 0.1318423039014436
Completed 14/10000 epoch, loss = 0.13045551936625732
Completed 15/10000 epoch, loss = 0.1290830606829166
Completed 16/10000 epoch, loss = 0.12773250109985093
Completed 17/10000 epoch, loss = 0.1264115933726222
Completed 18/10000 epoch, loss = 0.12512809883747275
Completed 19/10000 epoch, loss = 0.12388959968006986
Completed 

In [17]:
for x in X:
    print(xor_model.predict(x))

[[0.04307301]]
[[0.95559956]]
[[0.9597943]]
[[0.03697932]]
